# Cabecera

**Nombre completo del estudiante**: Valencia Hernandez Kevin Guadalupe
**Grupo**: 5BV1
**Carrera**: Ingenieria en Inteligencia Artificial
**Fecha de última modificación**: 12/05/25

---

## Descripción detallada del programa

Este programa implementa diferentes algoritmos para identificar similitudes entre palabras, frases y documentos, utilizando tanto enfoques semánticos como sintácticos. Se analizan cinco documentos de Project Gutenberg mediante varias técnicas:

1. **Similitud de palabras con synsets**: Uso de las métricas de WordNet "wup_similarity" y "path_similarity" para encontrar términos similares.
2. **Similitud de documentos con synsets**: Comparación de frases representativas mediante "path_similarity".
3. **Similitud de palabras con embedding**: Utilización del modelo pre-entrenado GloVe para identificar términos similares mediante similitud de coseno.
4. **Similitud de documentos con embedding**: Aplicación del modelo BERT para encontrar similitud entre documentos.

### Datos de entrada:
- **Textos de entrada**: Cinco introducciones de libros de Project Gutenberg de temáticas similares.
- **Modelos pre-entrenados**: GloVe y BERT para análisis de similitud.

---

## PRÁCTICA 4: IDENTIFICACIÓN DE PALABRAS, FRASES, Y DOCUMENTOS SIMILARES

In [ ]:
# Importaciones mínimas necesarias para comenzar
import nltk
import re
import string
from collections import Counter
from urllib.request import urlopen
import requests

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import wordnet as wn

try:
    import spacy
    try:
        nlp = spacy.load("en_core_web_sm")
    except:
        print("El modelo de spaCy no está instalado. Ejecuta: python -m spacy download en_core_web_sm")
except ImportError:
    print("spaCy no está instalado. Algunas funcionalidades no estarán disponibles.")

import random
random.seed(42)

print("Importaciones básicas completadas con éxito")

## 1. Generación de cuerpo de documentos

En esta sección, identificamos y descargamos cinco libros de interés desde Project Gutenberg. Seleccionamos obras de ciencia ficción clásica para mantener una temática similar. De cada libro, extraemos específicamente la introducción para formar nuestro corpus de análisis.

In [ ]:
import requests

def obtenerIntroduccionLibro1():
    url = "https://www.gutenberg.org/cache/epub/43713/pg43713.txt"
    response = requests.get(url)
    texto = response.text
    
    marcadorInicio = "INTRODUCTORY."
    marcadorFin = "PART I."
    
    indiceInicio = texto.find(marcadorInicio)
    if indiceInicio == -1:
        return "No se encontró la introducción"
    
    indiceInicio += len(marcadorInicio)
    indiceFin = texto.find(marcadorFin, indiceInicio)
    
    if indiceFin == -1:
        return "No se encontró el fin de la introducción"
    
    return texto[indiceInicio:indiceFin].strip()

In [ ]:
intro1 = obtenerIntroduccionLibro1()
print(intro1)

In [ ]:
def obtenerIntroduccionLibro2():
    url = "https://www.gutenberg.org/cache/epub/49211/pg49211.txt"
    response = requests.get(url)
    texto = response.text
    
    marcadorInicio = "INTRODUCTORY CHAPTER"
    marcadorFin = "CHAPTER I"
    
    indiceInicio = texto.find(marcadorInicio)
    if indiceInicio == -1:
        return "No se encontró la introducción"
    
    indiceInicio += len(marcadorInicio)
    indiceFin = texto.find(marcadorFin, indiceInicio)
    
    if indiceFin == -1:
        return "No se encontró el fin de la introducción"
    
    return texto[indiceInicio:indiceFin].strip()

In [ ]:
intro2 = obtenerIntroduccionLibro2()
print(intro2)

In [ ]:
import re

def obtenerIntroduccionLibro3():
    url = "https://www.gutenberg.org/cache/epub/16410/pg16410.txt"
    response = requests.get(url)
    texto = response.text
    
    # Usar regex con grupos de captura para obtener el texto entre INTRODUCTION y CHAPTER II
    # El primer grupo captura "CHAPTER I\n\nINTRODUCTION"
    # El segundo grupo captura todo el contenido hasta antes de "CHAPTER II"
    patron = r"(CHAPTER I\s*\n\s*\n\s*INTRODUCTION\s*\n\s*\n)([\s\S]*?)(?=\s*CHAPTER II)"
    
    coincidencia = re.search(patron, texto)
    
    if coincidencia:
        # Devolver solo el segundo grupo, que contiene el texto de la introducción
        return coincidencia.group(2).strip()
    else:
        return "No se encontró la introducción"

In [ ]:
intro3 = obtenerIntroduccionLibro3()
print(intro3)

In [ ]:
def obtenerIntroduccionLibro4():
    url = "https://www.gutenberg.org/cache/epub/48010/pg48010.txt"
    response = requests.get(url)
    texto = response.text
    
    # Buscar la cadena "INTRODUCTION" seguida por cualquier cosa hasta "CHAPTER I"
    primera_intro = texto.find("INTRODUCTION")
    if primera_intro == -1:
        return "No se encontró INTRODUCTION"
    
    # Buscar la segunda aparición de "INTRODUCTION"
    segunda_intro = texto.find("INTRODUCTION", primera_intro + 1)
    if segunda_intro == -1:
        return "No se encontró la segunda ocurrencia de INTRODUCTION"
    
    # Avanzar después de la palabra "INTRODUCTION"
    inicio_contenido = segunda_intro + len("INTRODUCTION")
    
    # Buscar CHAPTER I 
    fin_pos = texto.find("CHAPTER I", inicio_contenido)
    if fin_pos == -1:
        return "No se encontró CHAPTER I después de la segunda INTRODUCTION"
    
    # Extraer el texto entre inicio_contenido y fin_pos
    introduccion_texto = texto[inicio_contenido:fin_pos].strip()
    
    return introduccion_texto

In [ ]:
intro4 = obtenerIntroduccionLibro4()
print(intro4)

In [ ]:
def obtenerIntroduccionLibro5():
    url = "https://www.gutenberg.org/cache/epub/26163/pg26163.txt"
    response = requests.get(url)
    texto = response.text
    
    # Buscar la primera ocurrencia de "INTRODUCTION"
    primera_intro = texto.find("INTRODUCTION")
    if primera_intro == -1:
        return "No se encontró INTRODUCTION"
    
    # Buscar la segunda aparición de "INTRODUCTION"
    segunda_intro = texto.find("INTRODUCTION", primera_intro + 1)
    if segunda_intro == -1:
        return "No se encontró la segunda ocurrencia de INTRODUCTION"
    
    # Avanzar después de la palabra "INTRODUCTION"
    inicio_contenido = segunda_intro + len("INTRODUCTION")
    
    # Buscar CHAPTER I
    fin_pos = texto.find("CHAPTER I", inicio_contenido)
    if fin_pos == -1:
        return "No se encontró CHAPTER I después de la segunda INTRODUCTION"
    
    # Extraer el texto entre inicio_contenido y fin_pos
    introduccion_texto = texto[inicio_contenido:fin_pos].strip()
    
    return introduccion_texto

In [ ]:
intro5 = obtenerIntroduccionLibro5()
print(intro5)

In [ ]:
cuerpo = [intro1, intro2, intro3, intro4, intro5]

## 2. Normalización de documentos

En esta sección, realizamos la normalización y el preprocesamiento de cada documento del corpus. El proceso incluye:

1. Segmentación en oraciones
2. Tokenización de palabras
3. Etiquetado gramatical (POS tagging)
4. Aplicación de técnicas específicas de normalización

Estas técnicas de normalización se adaptan según los objetivos de cada análisis:
- Para similitud de palabras con synsets, necesitamos mantener la estructura de las oraciones y categorizar por POS
- Para similitud con embeddings, necesitamos eliminar stopwords y realizar lematización

In [ ]:
import spacy
import re
from nltk.tokenize import sent_tokenize
import string

# Cargar el modelo de spaCy
nlp = spacy.load("en_core_web_sm")

def normalizarDocumento(texto, tipo_normalizacion="general"):
    """
    Normaliza un documento segmentándolo en oraciones y tokenizándolo con etiquetas gramaticales,
    usando spaCy para el procesamiento.
    
    Args:
        texto (str): Texto a normalizar
        tipo_normalizacion (str): Tipo de normalización a aplicar ('general', 'synsets', 'embeddings')
        
    Returns:
        dict: Diccionario con el texto normalizado según diferentes niveles
    """
    resultado = {
        "oraciones_originales": [],
        "oraciones_tokenizadas": [],
        "oraciones_etiquetadas": [],
        "oraciones_normalizadas": []
    }
    
    # Limpieza previa del texto completo
    # Reemplazar saltos de línea y otros caracteres especiales con espacios
    texto = re.sub(r'[\r\n\t]+', ' ', texto)
    # Reemplazar múltiples espacios con uno solo
    texto = re.sub(r'\s+', ' ', texto).strip()
    
    # 1. Segmentación en oraciones (usando NLTK para mantener compatibilidad)
    oraciones = sent_tokenize(texto)
    resultado["oraciones_originales"] = oraciones
    
    for oracion in oraciones:
        # Limpieza adicional de la oración
        oracion = oracion.strip()
        
        # Procesar con spaCy
        doc = nlp(oracion)
        
        # 2. Tokenización (sin puntuación ni números)
        tokens = [token.text for token in doc 
                 if not token.is_punct and not token.like_num]
        resultado["oraciones_tokenizadas"].append(tokens)
        
        # 3. Etiquetado gramatical
        tokens_etiquetados = [(token.text, token.pos_) for token in doc 
                             if not token.is_punct and not token.like_num]
        resultado["oraciones_etiquetadas"].append(tokens_etiquetados)
        
        # 4. Normalización específica según el tipo
        if tipo_normalizacion == "synsets":
            # Para synsets: eliminar stopwords pero mantener la forma original
            tokens_normalizados = [token.text for token in doc 
                                  if not token.is_punct and not token.like_num 
                                  and not token.is_stop]
            
        elif tipo_normalizacion == "embeddings":
            # Para embeddings: eliminar stopwords y lematizar
            tokens_normalizados = [token.lemma_ for token in doc 
                                  if not token.is_punct and not token.like_num 
                                  and not token.is_stop]
        else:  # general
            # Normalización general: eliminar stopwords
            tokens_normalizados = [token.text for token in doc 
                                  if not token.is_punct and not token.like_num 
                                  and not token.is_stop]
        
        resultado["oraciones_normalizadas"].append(tokens_normalizados)
    
    return resultado

In [ ]:
# Normalización de la introducción 1
print("Normalizando introducción 1...")
intro1_normalizada = normalizarDocumento(intro1)

# Mostrar estadísticas
print(f"Oraciones originales: {len(intro1_normalizada['oraciones_originales'])}")
print(f"Tokens totales: {sum(len(tokens) for tokens in intro1_normalizada['oraciones_tokenizadas'])}")
print(f"Tokens después de normalización: {sum(len(tokens) for tokens in intro1_normalizada['oraciones_normalizadas'])}")

# Mostrar ejemplo de normalización completa para la primera oración
print("\nEjemplo completo con la primera oración:")
if intro1_normalizada['oraciones_originales']:
    primera_oracion = intro1_normalizada['oraciones_originales'][0]
    print(f"\nOriginal: {primera_oracion}")
    
    print("\nTokenizada:")
    print(intro1_normalizada['oraciones_tokenizadas'][0])
    
    print("\nEtiquetada gramaticalmente:")
    print(intro1_normalizada['oraciones_etiquetadas'][0])
    
    print("\nNormalizada (sin stopwords):")
    print(intro1_normalizada['oraciones_normalizadas'][0])

In [ ]:
# Normalización de la introducción 1
print("Normalizando introducción 1...")
intro1_normalizada = normalizarDocumento(intro1)

# Mostrar estadísticas
print(f"Oraciones originales: {len(intro1_normalizada['oraciones_originales'])}")
print(f"Tokens totales: {sum(len(tokens) for tokens in intro1_normalizada['oraciones_tokenizadas'])}")
print(f"Tokens después de normalización: {sum(len(tokens) for tokens in intro1_normalizada['oraciones_normalizadas'])}")

# Mostrar ejemplo de normalización completa para la primera oración
print("\nEjemplo completo con la primera oración:")
if intro1_normalizada['oraciones_originales']:
    primera_oracion = intro1_normalizada['oraciones_originales'][0]
    print(f"\nOriginal: {primera_oracion}")
    
    print("\nTokenizada:")
    print(intro1_normalizada['oraciones_tokenizadas'][0])
    
    print("\nEtiquetada gramaticalmente:")
    print(intro1_normalizada['oraciones_etiquetadas'][0])
    
    print("\nNormalizada (sin stopwords):")
    print(intro1_normalizada['oraciones_normalizadas'][0])

In [ ]:
cuerpo = [intro1, intro2, intro3, intro4, intro5]

In [ ]:
# Normalizar los 5 documentos para synsets y embeddings
documentos_synsets = []
documentos_embeddings = []

for i, doc in enumerate(cuerpo):
    print(f"Normalizando documento {i+1}...")
    
    # Normalización para synsets
    doc_synsets = normalizarDocumento(doc, tipo_normalizacion="synsets")
    documentos_synsets.append(doc_synsets)
    
    # Normalización para embeddings
    doc_embeddings = normalizarDocumento(doc, tipo_normalizacion="embeddings")
    documentos_embeddings.append(doc_embeddings)
    
    # Mostrar estadísticas básicas
    num_oraciones = len(doc_synsets["oraciones_originales"])
    num_tokens = sum(len(tokens) for tokens in doc_synsets["oraciones_tokenizadas"])
    num_tokens_normalizados = sum(len(tokens) for tokens in doc_synsets["oraciones_normalizadas"])
    
    print(f"  - Documento {i+1}: {num_oraciones} oraciones, {num_tokens} tokens")
    print(f"  - Tokens después de normalización para synsets: {num_tokens_normalizados}")
    print(f"  - Tokens después de normalización para embeddings: {sum(len(tokens) for tokens in doc_embeddings['oraciones_normalizadas'])}")
    print()

## 3. Similitud de palabras con synsets

En esta sección, utilizaremos WordNet para encontrar palabras similares a los verbos y sustantivos más comunes de cada documento. Se emplearán dos métricas de similitud diferentes:

1. **Wu-Palmer Similarity (wup_similarity)**: Mide la similitud basada en la profundidad relativa de dos synsets en la taxonomía de WordNet.
2. **Path Similarity (path_similarity)**: Mide la similitud basada en la distancia del camino más corto entre dos synsets.

Para cada documento, identificaremos:
- El verbo más común
- El sustantivo más común
- Los 5 términos más similares a cada uno según ambas métricas

In [ ]:
from collections import Counter
import nltk
from nltk.corpus import wordnet as wn

# Descargar el recurso de WordNet si es necesario
nltk.download('wordnet')

def encontrarPalabrasFrecuentes(documento_normalizado):
    """
    Encuentra los verbos y sustantivos más comunes en un documento.
    
    Args:
        documento_normalizado (dict): Documento normalizado con etiquetas POS
        
    Returns:
        tuple: (verbo_más_común, sustantivo_más_común)
    """
    # Contador para verbos y sustantivos
    verbos = Counter()
    sustantivos = Counter()
    
    # Analizar cada oración etiquetada
    for oracion in documento_normalizado["oraciones_etiquetadas"]:
        for palabra, pos in oracion:
            # En spaCy, 'VERB' para verbos y 'NOUN' para sustantivos
            if pos == 'VERB':
                verbos[palabra.lower()] += 1
            elif pos == 'NOUN':
                sustantivos[palabra.lower()] += 1
    
    # Encontrar los más comunes
    verbo_comun = verbos.most_common(1)
    sustantivo_comun = sustantivos.most_common(1)
    
    # Devolver los más comunes, o None si no hay
    verbo_mas_comun = verbo_comun[0][0] if verbo_comun else None
    sustantivo_mas_comun = sustantivo_comun[0][0] if sustantivo_comun else None
    
    return verbo_mas_comun, sustantivo_mas_comun

def obtenerSimilaresWordNet(palabra, pos, metrica='wup', top_n=5):
    """
    Obtiene las palabras más similares a una dada usando WordNet.
    
    Args:
        palabra (str): Palabra de referencia
        pos (str): Categoría gramatical ('n' para sustantivo, 'v' para verbo)
        metrica (str): Métrica de similitud ('wup' o 'path')
        top_n (int): Número de palabras similares a devolver
        
    Returns:
        list: Lista de tuplas (palabra, puntuación)
    """
    # Convertir pos de spaCy a formato WordNet
    if pos == 'VERB':
        pos_wn = 'v'
    elif pos == 'NOUN':
        pos_wn = 'n'
    else:
        pos_wn = 'n'  # Por defecto, sustantivo
    
    # Obtener synsets de la palabra
    synsets = wn.synsets(palabra, pos=pos_wn)
    if not synsets:
        return []
    
    # Usar el primer synset (más común)
    synset = synsets[0]
    
    # Obtener todos los synsets de la misma categoría
    todos_synsets = list(wn.all_synsets(pos=pos_wn))
    
    # Calcular similitud con cada synset
    similitudes = []
    for otro_synset in todos_synsets:
        # Evitar comparar con el mismo synset
        if otro_synset == synset:
            continue
        
        # Calcular similitud según la métrica especificada
        if metrica == 'wup':
            sim = synset.wup_similarity(otro_synset)
        else:  # path
            sim = synset.path_similarity(otro_synset)
        
        if sim:
            # Obtener el nombre de la palabra del synset (lema)
            nombre = otro_synset.lemma_names()[0]
            similitudes.append((nombre, sim))
    

    similitudes.sort(key=lambda x: x[1], reverse=True)
    return similitudes[:top_n]

In [ ]:
# Encontrar palabras frecuentes en el documento 1
verbo_comun, sustantivo_comun = encontrarPalabrasFrecuentes(documentos_synsets[0])

print(f"Documento 1:")
print(f"Verbo más común: {verbo_comun}")
print(f"Sustantivo más común: {sustantivo_comun}")

# Encontrar palabras similares usando Wu-Palmer Similarity
print("\nPalabras similares al verbo más común usando Wu-Palmer Similarity:")
similares_wup = obtenerSimilaresWordNet(verbo_comun, 'VERB', 'wup')
for palabra, puntuacion in similares_wup:
    print(f"  - {palabra}: {puntuacion:.4f}")

print("\nPalabras similares al sustantivo más común usando Wu-Palmer Similarity:")
similares_wup = obtenerSimilaresWordNet(sustantivo_comun, 'NOUN', 'wup')
for palabra, puntuacion in similares_wup:
    print(f"  - {palabra}: {puntuacion:.4f}")

# Encontrar palabras similares usando Path Similarity
print("\nPalabras similares al verbo más común usando Path Similarity:")
similares_path = obtenerSimilaresWordNet(verbo_comun, 'VERB', 'path')
for palabra, puntuacion in similares_path:
    print(f"  - {palabra}: {puntuacion:.4f}")

print("\nPalabras similares al sustantivo más común usando Path Similarity:")
similares_path = obtenerSimilaresWordNet(sustantivo_comun, 'NOUN', 'path')
for palabra, puntuacion in similares_path:
    print(f"  - {palabra}: {puntuacion:.4f}")

In [ ]:
# Análisis de similitud para todos los documentos
resultados_similitud = []

for i, doc_normalizado in enumerate(documentos_synsets):
    print(f"\n{'='*50}")
    print(f"Documento {i+1}")
    print(f"{'='*50}")
    
    # Encontrar palabras frecuentes
    verbo_comun, sustantivo_comun = encontrarPalabrasFrecuentes(doc_normalizado)
    
    print(f"Verbo más común: {verbo_comun}")
    print(f"Sustantivo más común: {sustantivo_comun}")
    
    # Resultados para este documento
    resultado_doc = {
        'verbo': verbo_comun,
        'sustantivo': sustantivo_comun,
        'verbo_wup': [],
        'verbo_path': [],
        'sustantivo_wup': [],
        'sustantivo_path': []
    }
    
    # Similitud para el verbo
    if verbo_comun:
        # Wu-Palmer Similarity
        print("\nPalabras similares al verbo usando Wu-Palmer Similarity:")
        similares_wup = obtenerSimilaresWordNet(verbo_comun, 'VERB', 'wup')
        for palabra, puntuacion in similares_wup:
            print(f"  - {palabra}: {puntuacion:.4f}")
            resultado_doc['verbo_wup'].append((palabra, puntuacion))
        
        # Path Similarity
        print("\nPalabras similares al verbo usando Path Similarity:")
        similares_path = obtenerSimilaresWordNet(verbo_comun, 'VERB', 'path')
        for palabra, puntuacion in similares_path:
            print(f"  - {palabra}: {puntuacion:.4f}")
            resultado_doc['verbo_path'].append((palabra, puntuacion))
    
    # Similitud para el sustantivo
    if sustantivo_comun:
        # Wu-Palmer Similarity
        print("\nPalabras similares al sustantivo usando Wu-Palmer Similarity:")
        similares_wup = obtenerSimilaresWordNet(sustantivo_comun, 'NOUN', 'wup')
        for palabra, puntuacion in similares_wup:
            print(f"  - {palabra}: {puntuacion:.4f}")
            resultado_doc['sustantivo_wup'].append((palabra, puntuacion))
        
        # Path Similarity
        print("\nPalabras similares al sustantivo usando Path Similarity:")
        similares_path = obtenerSimilaresWordNet(sustantivo_comun, 'NOUN', 'path')
        for palabra, puntuacion in similares_path:
            print(f"  - {palabra}: {puntuacion:.4f}")
            resultado_doc['sustantivo_path'].append((palabra, puntuacion))
    
    resultados_similitud.append(resultado_doc)

## 4. Similitud de documentos con synsets

En esta sección, extraeremos la frase más representativa de cada documento usando el algoritmo RAKE (Rapid Automatic Keyword Extraction), que según los resultados de la práctica 3 demostró ser uno de los más efectivos para la extracción de frases clave.

Luego, seleccionaremos el primer documento como base y compararemos su frase representativa con las de los otros cuatro documentos utilizando la métrica Path_similarity de WordNet.

In [ ]:
# Importamos las bibliotecas necesarias para la extracción de frases clave usando RAKE
import nltk
from nltk.corpus import wordnet as wn
import numpy as np
from rake_nltk import Rake

# Verificamos que se hayan descargado los recursos necesarios
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

def extraer_frase_representativa(texto, num_frases=1):
    """
    Extrae las frases más representativas de un texto usando RAKE.

    Args:
        texto (str): Texto de entrada
        num_frases (int): Número de frases a extraer

    Returns:
        list: Lista de frases ordenadas por relevancia
    """
    # Inicializar RAKE
    rake = Rake(
        stopwords=nltk.corpus.stopwords.words('english'),
        include_repeated_phrases=False
    )

    # Extraer frases clave
    rake.extract_keywords_from_text(texto)
    frases_clave = rake.get_ranked_phrases()

    # Devolver las frases más relevantes (ordenadas por puntuación)
    return frases_clave[:num_frases] if frases_clave else []

# Extraer la frase más representativa de cada documento
frases_representativas = []
for i, doc in enumerate(cuerpo):
    print(f"Documento {i+1}:")
    frases = extraer_frase_representativa(doc)
    frase = frases[0] if frases else "No se encontró frase representativa"
    frases_representativas.append(frase)
    print(f"Frase representativa: {frase}\n")

In [ ]:
def comparar_similitud_documento(doc_base_idx, frases_representativas):
    """
    Compara la similitud entre la frase representativa de un documento base
    y las frases representativas de otros documentos usando Path Similarity.

    Args:
        doc_base_idx (int): Índice del documento base
        frases_representativas (list): Lista de frases representativas de cada documento

    Returns:
        list: Lista de tuplas (índice_documento, puntuación_similitud)
    """
    resultado = []
    frase_base = frases_representativas[doc_base_idx]

    # Tokenizar la frase base
    tokens_base = word_tokenize(frase_base.lower())

    # Filtrar solo sustantivos y verbos (más relevantes para comparación semántica)
    pos_tags_base = pos_tag(tokens_base)
    palabras_base = [palabra for palabra, tag in pos_tags_base
                    if tag.startswith('NN') or tag.startswith('VB')]

    # Obtener synsets para palabras base
    synsets_base = []
    for palabra in palabras_base:
        synsets = wn.synsets(palabra)
        if synsets:
            synsets_base.append(synsets[0])

    print(f"\nComparando documento {doc_base_idx+1} con los demás:")

    for i, frase in enumerate(frases_representativas):
        if i == doc_base_idx:
            continue

        # Tokenizar la frase a comparar
        tokens = word_tokenize(frase.lower())

        # Filtrar solo sustantivos y verbos
        pos_tags = pos_tag(tokens)
        palabras = [palabra for palabra, tag in pos_tags
                   if tag.startswith('NN') or tag.startswith('VB')]

        # Obtener synsets
        synsets_doc = []
        for palabra in palabras:
            synsets = wn.synsets(palabra)
            if synsets:
                synsets_doc.append(synsets[0])

        # Calcular similitud (promedio de similitudes máximas entre synsets)
        if synsets_base and synsets_doc:
            similitudes = []
            for s1 in synsets_base:
                max_sim = 0
                for s2 in synsets_doc:
                    sim = s1.path_similarity(s2)
                    if sim and sim > max_sim:
                        max_sim = sim
                if max_sim > 0:
                    similitudes.append(max_sim)

            # Promedio de similitudes
            similitud_promedio = sum(similitudes) / len(similitudes) if similitudes else 0
            resultado.append((i, similitud_promedio))
            print(f"Documento {i+1}: Similitud {similitud_promedio:.4f}")
        else:
            resultado.append((i, 0))
            print(f"Documento {i+1}: No se pudo calcular similitud (sin synsets)")

    return resultado

In [ ]:
# Comparar documento 1 con los demás
similitudes_doc1 = comparar_similitud_documento(0, frases_representativas)

In [ ]:
print("Frases representativas extraídas:")
for i, frase in enumerate(frases_representativas):
    print(f"Documento {i+1}: {frase}")

## 5. Similitud de palabras con "embedding"

En esta sección, utilizaremos el modelo pre-entrenado GloVe (Global Vectors for Word Representation) "Wikipedia 2014 + Gigaword 5" para identificar en cada documento los 5 términos más similares al verbo más frecuente.

Para calcular la similitud entre términos, usaremos la medida de similitud de coseno, que es particularmente efectiva para embeddings de palabras.

In [ ]:
import numpy as np
import os
import requests
import zipfile
from sklearn.metrics.pairwise import cosine_similarity

def cargar_glove(dim=100):
    """
    Descarga y carga vectores GloVe "Wikipedia 2014 + Gigaword 5".
    
    Args:
        dim (int): Dimensión de los vectores (50, 100, 200 o 300)
        
    Returns:
        dict: Diccionario con palabras como claves y vectores como valores
    """
    # URL del modelo GloVe
    url = "https://nlp.stanford.edu/data/glove.6B.zip"
    
    # Nombre del archivo
    zip_filename = "glove.6B.zip"
    txt_filename = f"glove.6B.{dim}d.txt"
    
    # Verificar si ya existe el archivo extraído
    if not os.path.exists(txt_filename):
        # Verificar si existe el zip
        if not os.path.exists(zip_filename):
            print(f"Descargando {zip_filename} desde {url}...")
            print("Este proceso puede tardar varios minutos dependiendo de tu conexión.")
            r = requests.get(url)
            with open(zip_filename, 'wb') as f:
                f.write(r.content)
        
        # Extraer el archivo
        print(f"Extrayendo {txt_filename}...")
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extract(txt_filename)
    
    # Cargar vectores
    print(f"Cargando vectores GloVe de dimensión {dim}...")
    embeddings = {}
    with open(txt_filename, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            valores = line.split()
            palabra = valores[0]
            vector = np.array([float(val) for val in valores[1:]])
            embeddings[palabra] = vector
            
            # Para mostrar progreso
            if i % 100000 == 0:
                print(f"  Procesadas {i} líneas...")
    
    print(f"Cargados {len(embeddings)} vectores GloVe.")
    return embeddings

# Cargar vectores GloVe
try:
    glove_vectors = cargar_glove(dim=100)
except Exception as e:
    print(f"Error al cargar GloVe: {e}")
    print("Creando diccionario vacío como fallback.")
    glove_vectors = {}

In [ ]:
from collections import Counter

def encontrar_verbo_mas_frecuente(documento_normalizado):
    """
    Encuentra el verbo más frecuente en un documento normalizado.
    
    Args:
        documento_normalizado (dict): Documento normalizado
        
    Returns:
        str: Verbo más frecuente
    """
    contador_verbos = Counter()
    
    # Contar todos los verbos
    for oracion in documento_normalizado["oraciones_etiquetadas"]:
        for palabra, pos in oracion:
            if pos == 'VERB':
                contador_verbos[palabra.lower()] += 1
    
    # Obtener el verbo más frecuente
    if contador_verbos:
        return contador_verbos.most_common(1)[0][0]
    else:
        return None

# Encontrar el verbo más frecuente en cada documento
verbos_frecuentes = []

for i, doc in enumerate(documentos_synsets):
    verbo = encontrar_verbo_mas_frecuente(doc)
    verbos_frecuentes.append(verbo)
    print(f"Documento {i+1}: Verbo más frecuente -> '{verbo}'")

In [ ]:
def encontrar_similares_glove(palabra, embeddings, top_n=5):
    """
    Encuentra términos similares a una palabra usando embeddings GloVe y similitud de coseno.
    
    Args:
        palabra (str): Palabra de referencia
        embeddings (dict): Diccionario de embeddings GloVe
        top_n (int): Número de términos similares a devolver
        
    Returns:
        list: Lista de tuplas (palabra, similitud)
    """
    # Verificar si la palabra está en el vocabulario
    palabra = palabra.lower()
    if palabra not in embeddings:
        return []
    
    # Vector de la palabra de referencia
    vector = embeddings[palabra]
    vector = vector.reshape(1, -1)
    

    similitudes = []
    for otra_palabra, otro_vector in embeddings.items():
        if otra_palabra == palabra:
            continue
        
        otro_vector = otro_vector.reshape(1, -1)

        sim = cosine_similarity(vector, otro_vector)[0][0]
        similitudes.append((otra_palabra, sim))

    similitudes.sort(key=lambda x: x[1], reverse=True)
    return similitudes[:top_n]

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 12})

for i, verbo in enumerate(verbos_frecuentes):
    if verbo and verbo in glove_vectors:
        similares = encontrar_similares_glove(verbo, glove_vectors)
        
        if similares:

            plt.figure(figsize=(10, 6))

            palabras = [palabra for palabra, _ in similares]
            similitudes = [similitud for _, similitud in similares]

            plt.barh(range(len(palabras)), similitudes, align='center', color='skyblue')
            plt.yticks(range(len(palabras)), palabras)

            plt.xlabel('Similitud de coseno')
            plt.title(f'Documento {i+1}: 5 términos más similares a "{verbo}"')
            plt.xlim(0, 1)  # Escala de similitud de coseno: 0-1
            plt.grid(axis='x', linestyle='--', alpha=0.7)
            
            plt.tight_layout()
            plt.show()

## 6. Similitud de documentos con "embedding"

En esta sección, implementaremos la comparación de similitud entre documentos utilizando el modelo pre-entrenado BERT (Bidirectional Encoder Representations from Transformers). A diferencia del enfoque basado en synsets que usamos anteriormente, BERT captura el significado contextual completo de los documentos.

Utilizaremos específicamente el modelo "bert-base-uncased", que no hace distinción entre mayúsculas y minúsculas, para obtener embeddings de documentos y calcular su similitud mediante la similitud del coseno.

In [ ]:
try:
    import transformers
except ImportError:
    !pip install transformers

try:
    import torch
except ImportError:
    !pip install torch

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import time

print("Cargando modelo BERT 'bert-base-uncased'...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")
model = model.to(device)

In [ ]:
def obtener_embedding_bert(texto, max_length=512):
    """
    Obtiene el embedding de un documento usando BERT.

    Args:
        texto (str): Texto del documento
        max_length (int): Longitud máxima de tokens

    Returns:
        numpy.ndarray: Vector de embedding del documento
    """
    inputs = tokenizer(texto, return_tensors="pt", padding=True,
                      truncation=True, max_length=max_length)

    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()

    return embedding

embeddings_documentos = []
tiempos = []

print("Calculando embeddings BERT para cada documento...")
for i, doc in enumerate(cuerpo):
    inicio = time.time()
    print(f"Procesando documento {i+1}...")

    embedding = obtener_embedding_bert(doc)
    embeddings_documentos.append(embedding)

    fin = time.time()
    tiempo = fin - inicio
    tiempos.append(tiempo)
    print(f"  Tiempo: {tiempo:.2f} segundos")

In [ ]:
print("\nSimilitud entre documentos usando embeddings BERT:")
print(f"{'-'*50}")

num_docs = len(embeddings_documentos)
matriz_similitud = np.zeros((num_docs, num_docs))

for i in range(num_docs):
    for j in range(i, num_docs):

        sim = cosine_similarity(embeddings_documentos[i], embeddings_documentos[j])[0][0]
        matriz_similitud[i, j] = sim
        matriz_similitud[j, i] = sim

        if i != j:
            print(f"Documento {i+1} ↔ Documento {j+1}: {sim:.4f}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df_similitud = pd.DataFrame(matriz_similitud)
df_similitud.index = [f'Doc {i+1}' for i in range(num_docs)]
df_similitud.columns = [f'Doc {i+1}' for i in range(num_docs)]

plt.figure(figsize=(10, 8))
sns.heatmap(df_similitud, annot=True, cmap="YlGnBu", fmt=".2f", linewidths=.5)
plt.title('Similitud entre documentos usando BERT')
plt.tight_layout()
plt.show()

doc_base_idx = 0
similitudes_bert = []

for i in range(num_docs):
    if i != doc_base_idx:
        similitudes_bert.append((i, matriz_similitud[doc_base_idx, i]))

similitudes_bert.sort(key=lambda x: x[1], reverse=True)

print(f"Documento base ({doc_base_idx + 1})")
print("\nRanking de similitud (BERT):")
for i, sim in similitudes_bert:
    print(f"Documento {i+1}: {sim:.4f}")